<a href="https://colab.research.google.com/github/ngngocsonan2610/note/blob/master/1.PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[Preprocessing](#scrollTo=-2COeSc5ml1_)

>[Transformation Scaling/Standardize](#scrollTo=PcX1fpPrrg-_)

>[Encoding](#scrollTo=qeLaVzFW0Se6)

>>>[Onehot Encode](#scrollTo=HULldo6Zm6l1)

>>>[Label Encode](#scrollTo=Hz-vTHED0Vbc)

>>>[Reduce memory](#scrollTo=WzFvPys_F9KB)

>>>[Reduce dimensional checking](#scrollTo=a_a8z7m7uec_)

>>>>[PCA](#scrollTo=TQSNaBpvtFjT)

>>>>[T-sne](#scrollTo=kUzEGvostxdh)

>[Features Engineer](#scrollTo=ffkEEAsbC4EW)

>[Feature Selection](#scrollTo=LfdUhklbDAQa)

>>>[High correlation](#scrollTo=k9prSardKWXm)

>>>[WOE & IV](#scrollTo=t-8cO9PJ3ryk)

>>>[RFE](#scrollTo=cmnU0NRff_ck)

>[Data Type Handling](#scrollTo=JfMtLkkO7fxr)

>>>[Under-sampling](#scrollTo=5mzGc0BE7xWq)

>>>[Over-sampling](#scrollTo=4il7y5CI8O8D)

>>>>[RandomOverSampler](#scrollTo=2ioqMRx98Syz)

>>>>[SMOTE](#scrollTo=8A8WLw5B8dnc)

>[Validation](#scrollTo=cJ7_EPQZIm0I)



# Transformation Scaling/Standardize
News:
- https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing


Căn bản:
- [Scale, Standardize, or Normalize with Scikit-Learn](https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02)
![alt text](https://miro.medium.com/max/4800/1*z-C9ANBC4rjsk-ZK4wzijg.png)
- [Why, How and When to Scale your Features](https://medium.com/greyatom/why-how-and-when-to-scale-your-features-4b30ab09db5e)


- Đối với từng model nên xét thêm





In [ ]:
from sklearn.preprocessing import RobustScaler
nottrans_num_col = ['AGSegCode', 'MAPASegCode',
                    'Leader_AGSegCode', 'Leader_MAPASegCode']
                    
scale_col = [c for c in WOE_numeric_col if c not in nottrans_num_col]
print('scale cols' ,scale_col)
robust_scale = RobustScaler().fit(round(df_WOE[scale_col],2))
df_WOE[scale_col] = robust_scale.transform(df_WOE[scale_col])

# Encoding


## Onehot Encode

In [ ]:
onehot_col = ['a', 'b', 'c']

def onehot_encode(df,onehot_col):
    dummies_col=list()
    for c in onehot_col:
        print('> Get dummies with prefix {}'.format(c+'_'))
        df_dummies = pd.get_dummies(df[c], prefix = c ) 
        dummies_col = dummies_col +(df_dummies.columns.to_list())
        df = pd.concat( [df, df_dummies] , axis = 1)
    print('dummies col: ',dummies_col)
    return df,dummies_col


temp,new_onehot_col = onehot_encode(temp,onehot_col)

## Label Encode

In [ ]:
df = df.replace({
             'Leader_AGLevel' : {'PM':1,'UM':2,'BM':3}
           })

In [ ]:
numeric_col = ['Leader_RegionCode', 'RegionCode', 'ContactProvince_Code','EducationCode']
for col in numeric_col:
    print('> ',col)
    df[col] = df[col].astype('category').cat.codes

In [ ]:
'''
BINs of numeric features
'''
# AVERAGE_INCOME
df_model['AVERAGE_INCOME_bin_LE'] = pd.qcut(df_model.AVERAGE_INCOME, q=7, precision=0).astype('category').cat.codes
bin_labels = []
for string in sorted(pd.qcut(round(df_model.AVERAGE_INCOME/10**6).astype('int'), q=7, precision=0).unique()):
    x = str(string).replace(r'(', '').replace(r']', '').replace(r', ', '-').replace(r'\.0', '')
    x = x +'m'
    bin_labels.append(x)
df_model['AVERAGE_INCOME_bin'] = pd.qcut(df_model.AVERAGE_INCOME, q=7, precision=0,labels=bin_labels).astype('str')  

### feature_engine.categorical_encoders 
CountFrequencyCategoricalEncoder
- https://feature-engine.readthedocs.io/en/latest/encoders/CountFrequencyCategoricalEncoder.html

In [ ]:
for col in cat_cols:
    train_df[col] = train_df[col].astype(str)
    test_df[col] = test_df[col].astype(str)
    
from feature_engine import categorical_encoders as ce

# Label-Encoding 
lbl = LabelEncoder()
for col in cat_cols:
    assert train_df[col].dtypes == test_df[col].dtypes
    
    encoder = ce.CountFrequencyCategoricalEncoder(encoding_method='frequency')
    
    train_df[col] = train_df[col].fillna('None')
    test_df[col] = test_df[col].fillna('None')
    # Only take the common values in Train/Test-set
    common_vals = list(set(train_df[col]).intersection(set(test_df[col])))
    
    # Take if vals appeared both 5 times
    common_vals = set(train_df[col].value_counts()[train_df[col].value_counts()> 20].index)\
                        .intersection(test_df[col].value_counts()[test_df[col].value_counts()>20].index)
    
    # Replace not-common values with "Missing" or np.NaN      
    train_df.loc[~train_df[col].isin(common_vals), col] = 'Missing'
    test_df.loc[~test_df[col].isin(common_vals), col] = 'Missing'

    # Implement LE
    lbl.fit(train_df[col].tolist() + test_df[col].tolist())
    train_df[col] = lbl.transform(train_df[col])
    test_df[col] = lbl.transform(test_df[col])

In [ ]:
## đổi tên
fe_cols = cat_cols
for col in fe_cols:
    temp_df = pd.concat([train_df[col], test_df[col]], axis=0)
    d_fe = temp_df.value_counts().to_dict()
    train_df[col+'_fe'] = train_df[col].apply(lambda x: d_fe.get(x))
    test_df[col+'_fe'] = test_df[col].apply(lambda x: d_fe.get(x))

### category_encoders

In [ ]:
import category_encoders as ce

In [ ]:
# Create the encoder
t = pd.concat([train, test]).reset_index(drop=True)
count_enc = ce.CountEncoder().fit_transform(t[cat_features])
tt = t.join(count_enc.add_suffix("_count"))

# phân train-test
f2_train = tt.loc[tt.index < train.shape[0]]
f2_test = tt.loc[tt.index >= train.shape[0]]

columns = sorted(set(f2_train.columns).intersection(f2_test.columns))
print(len(columns))

## Reduce memory

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

## Reduce dimensional checking


#### PCA
- [PCA in python & spark](https://dataplatform.cloud.ibm.com/analytics/notebooks/v2/02716668-5f99-40b1-b4a1-1a820b6e5c55/view?access_token=50797d4e6ec323c16603c628b41e1e3f203221faa3e826a218724558ed57de26)

In [ ]:
#scaling only on tranform data
df_V = df_V[c_V_trans] 
df_V_test = df_V_test[c_V_trans]

#standardize scaled
scaler = StandardScaler().fit(df_V)
df_V_scaled = scaler.transform(df_V)
print('Scaled mean df_V',df_V_scaled[:,0].mean())  # zero (or very close)
print('Scaled std df_V',df_V_scaled[:,0].std()) 
scaler = StandardScaler().fit(df_V_test)
df_V_test_scaled = scaler.transform(df_V_test)
print('Scaled mean df_V',df_V_test_scaled[:,0].mean())  # zero (or very close)
print('Scaled std df_V',df_V_test_scaled[:,0].std()) 

# plot cumulative explained variance
# pca = PCA().fit(df_V_scaled)
# plt.plot(np.cumsum(pca.explained_variance_ratio_))
# #plt.xlim(0,7,1)
# plt.xlabel('Number of components')
# plt.ylabel('Cumulative explained variance')


# Setup Principal component analysis
pca = PCA(n_components=125) 
#pca = PCA(n_components=0.96)  #v2: n_components=0.95
df_V_pca = pca.fit_transform(df_V_scaled)
np.save('df_V_pca_v3.npy',df_V_pca)
#df_V_pca.to_csv('df_V_pca', sep='\t')
df_V_test_pca = pca.fit_transform(df_V_test_scaled)
np.save('df_V_test_pca_v3.npy',df_V_test_pca)


In [ ]:
import itertools
import warnings
warnings.filterwarnings("ignore")
import io
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization

'''
Scatter plot giữa 3 cột bất kỳ trong dữ liệu, được hue = TARGET
cho cái nhìn về dữ liệu, xem liệu nó có khả năng phân tách không
'''

trace1 = go.Scatter3d(x = churn["MEMBER_ANNUAL_INCOME"],
                      y = churn["ANNUAL_FEES"],
                      z = churn["MEMBERSHIP_TERM_YEARS"],
                      mode = "markers",
                      name = "Churn customers",
                      text = "Id : " + churn["MEMBERSHIP_NUMBER"],
                      marker = dict(size = 1,color = "red")
                     )
trace2 = go.Scatter3d(x = not_churn["MEMBER_ANNUAL_INCOME"],
                      y = not_churn["ANNUAL_FEES"],
                      z = not_churn["MEMBERSHIP_TERM_YEARS"],
                      name = "Non churn customers",
                      text = "Id : " + not_churn["MEMBERSHIP_NUMBER"],
                      mode = "markers",
                      marker = dict(size = 1,color= "green")
                     )

layout = go.Layout(dict(title = "Monthly charges,total charges & tenure in customer attrition",
                        scene = dict(camera = dict(up=dict(x= 0 , y=0, z=0),
                                                   center=dict(x=0, y=0, z=0),
                                                   eye=dict(x=1.25, y=1.25, z=1.25)),
                                     xaxis  = dict(title = "annual incomes",
                                                   gridcolor='rgb(255, 255, 255)',
                                                   zerolinecolor='rgb(255, 255, 255)',
                                                   showbackground=True,
                                                   backgroundcolor='rgb(230, 230,230)'),
                                     yaxis  = dict(title = "annual fees",
                                                   gridcolor='rgb(255, 255, 255)',
                                                   zerolinecolor='rgb(255, 255, 255)',
                                                   showbackground=True,
                                                   backgroundcolor='rgb(230, 230,230)'
                                                  ),
                                     zaxis  = dict(title = "term years",
                                                   gridcolor='rgb(255, 255, 255)',
                                                   zerolinecolor='rgb(255, 255, 255)',
                                                   showbackground=True,
                                                   backgroundcolor='rgb(230, 230,230)'
                                                  )
                                    ),
                        height = 700,
                       )
                  )
                  

data = [trace1,trace2]
fig  = go.Figure(data = data,layout = layout)
py.iplot(fig)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)

X = tmp[[i for i in tmp.columns if i not in Id_col + target_col]]
Y = tmp[target_col + Id_col]

principal_components = pca.fit_transform(X)
pca_data = pd.DataFrame(principal_components,columns = ["PC1","PC2"])
pca_data = pca_data.merge(Y,left_index=True,right_index=True,how="left")
pca_data["CHURN"] = pca_data["CHURN"].replace({1:"CANCELLED",0:"INFORCE"})

def pca_scatter(target,color) :
    tracer = go.Scatter(x = pca_data[pca_data["CHURN"] == target]["PC1"] ,
                        y = pca_data[pca_data["CHURN"] == target]["PC2"],
                        name = target,mode = "markers",
                        marker = dict(color = color,
                                      line = dict(width = .5),
                                      symbol =  "diamond-open"),
                        text = ("Customer Id : " + 
                                pca_data[pca_data["CHURN"] == target]['MEMBERSHIP_NUMBER'])
                       )
    return tracer

layout = go.Layout(dict(title = "Visualising data with principal components",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "principal component 1",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "principal component 2",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        height = 600
                       )
                  )
trace1 = pca_scatter("CANCELLED",'red')
trace2 = pca_scatter("INFORCE",'royalblue')
data = [trace2,trace1]
fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

In [ ]:
from sklearn.datasets import make_blobs
from sklearn import datasets
class PCA:
    def __init__(self, n_components):
        self.n_components = n_components
        self.components = None
        self.mean = None

    def fit(self, X):
        self.mean = np.mean(X, axis=0)
        X = X - self.mean
        cov = np.cov(X.T)

        evalue, evector = np.linalg.eig(cov)

        eigenvectors = evector.T
        idxs = np.argsort(evalue)[::-1]

        evalue = evalue[idxs]
        evector = evector[idxs]
        self.components = evector[0:self.n_components]

    def transform(self, X):
        #project data
        X = X - self.mean
        return(np.dot(X, self.components.T))

           


In [ ]:
from sklearn.preprocessing import RobustScaler        
scale_col = ['A', 'B', 'C']
print('scale cols: ' ,scale_col)
robust_scale = RobustScaler().fit(round(data[scale_col],2))
data[scale_col] = robust_scale.transform(data[scale_col])

X = data[col]
y = data['label']

pca = PCA(2)
pca.fit(X)
X_projected = pca.transform(X)

In [ ]:
'''
2d plot
'''
x1 = X_projected[:,0]
x2 = X_projected[:,1]
plt.figure(figsize=(20,10))
plt.scatter(x1,x2,c=y,edgecolor='none',alpha=0.8,cmap=plt.cm.get_cmap('viridis',2),s=6)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar()
plt.margins(0.05)
plt.show()

In [ ]:
'''
3d plot
'''
from mpl_toolkits import mplot3d

X = data[col]
y = data['label']

pca = PCA(3)
pca.fit(X)
X_projected = pca.transform(X)

ax = plt.figure(figsize=(16,10)).gca(projection='3d')
ax.scatter(
    xs=X_projected[:,0], 
    ys=X_projected[:,1], 
    zs=X_projected[:,2], 
    c=y, 
    cmap=plt.cm.get_cmap('viridis',2)#'tab10'
    ,s=6
)
ax.set_xlabel('pca-one')
ax.set_ylabel('pca-two')
ax.set_zlabel('pca-three')
ax.margins(0.05)
plt.show()

#### T-sne
- [VisualizingDatausingt-SNE](http://jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf)


In [ ]:
from sklearn.manifold import TSNE

import time

In [ ]:
from sklearn.preprocessing import RobustScaler        
scale_col = ['A', 'B', 'C']
print('scale cols: ' ,scale_col)
robust_scale = RobustScaler().fit(round(data[scale_col],2))
data[scale_col] = robust_scale.transform(data[scale_col])

X = data[col]
y = data['label']

'''
reduce to 2d & plot
'''
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(X)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

data['tsne-2d-one'] = tsne_results[:,0]
data['tsne-2d-two'] = tsne_results[:,1]

plt.figure(figsize=(16,10))

sns.scatterplot(
    x=tsne_results[:,0], y= tsne_results[:,1],
    hue=data['label'],
    palette=sns.color_palette("hls", 2),
    #data=data,
    legend="full",
    alpha=0.6)


In [ ]:
'''
reduce to 3d & plot
'''
X = data[col]
y = data['label']

time_start = time.time()
tsne = TSNE(n_components=3, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(X)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

ax = plt.figure(figsize=(16,10)).gca(projection='3d')
ax.scatter(
    xs=tsne_results[:,0], 
    ys=tsne_results[:,1], 
    zs=tsne_results[:,2], 
    c=data['label'], 
    cmap=plt.cm.get_cmap('viridis',2)#'tab10'
)
ax.set_xlabel('pca-one')
ax.set_ylabel('pca-twp')
ax.set_zlabel('pca-three')

plt.show()

# Features Engineer




[Simple FE](https://machinelearningcoban.com/general/2017/02/06/featureengineering/)
-    Trực tiếp lấy raw data
-    Bag-of-words
  -    Bag-of-Words trong Computer Vision
-    Feature Scaling and Normalization
  -        Rescaling
  -        Standardization
  -        Scaling to unit length

  - Feature Generation: ở bước này tập trung các kĩ thuật để tạo ra feature: xử lí nlp, image, binning, scaling, grouping, aggregate.
  - Kết hợp với Modelling để tìm ra nhóm feature tốt và tập trung mạnh vào đó

## numeric

In [ ]:
'''
một số FE cho prices và các biến numeric quan trọng
PRICES
'''

print('Prices')

# We can do some basic aggregations
prices_df['price_max'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('max')
prices_df['price_min'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('min')
prices_df['price_std'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('std')
prices_df['price_mean'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('mean')

# and do price normalization (min/max scaling)
prices_df['price_norm'] = prices_df['sell_price']/prices_df['price_max']

# Some items are can be inflation dependent
# and some items are very "stable"
prices_df['price_nunique'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('nunique')
prices_df['item_nunique'] = prices_df.groupby(['store_id','sell_price'])['item_id'].transform('nunique')

# I would like some "rolling" aggregations
# but would like months and years as "window"
calendar_prices = calendar_df[['wm_yr_wk','month','year']]
calendar_prices = calendar_prices.drop_duplicates(subset=['wm_yr_wk'])
prices_df = prices_df.merge(calendar_prices[['wm_yr_wk','month','year']], on=['wm_yr_wk'], how='left')
del calendar_prices

# Now we can add price "momentum" (some sort of)
# Shifted by week # by month mean # by year mean
prices_df['price_momentum'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
prices_df['price_momentum_7'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(7))
prices_df['price_momentum_14'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(14))
prices_df['price_momentum_m'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
prices_df['price_momentum_y'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')

del prices_df['month'], prices_df['year']

## data-time

In [ ]:
'''
TÌM HOLIDAYS
https://github.com/dr-prodigy/python-holidays
'''
calendar_prices = calendar_df[['wm_yr_wk','date','month','year']]
#calendar_prices = calendar_prices.drop_duplicates(subset=['wm_yr_wk'])
## get holidays
north_america = holidays.US() + holidays.MX() 
calendar_prices['holiday'] = calendar_prices.date.apply(lambda x : north_america.get(x))

calendar_prices['nearest_holiday'] = pd.to_datetime(pd.Series(np.where(calendar_prices.holiday.isnull()==False, pd.to_numeric(pd.to_datetime(calendar_prices.date)), calendar_prices.holiday)).astype(float).interpolate(method ='nearest'))

calendar_prices.iloc[:7,].nearest_holiday.fillna(calendar_prices.nearest_holiday[8], inplace=True)

calendar_prices.iloc[1949:,].nearest_holiday.fillna(calendar_prices.nearest_holiday[1948], inplace=True)
#calendar_prices['tw_holiday'] = pd.to_datetime(pd.Series(np.where(calendar_prices.holiday.isnull()==False, pd.to_numeric(pd.to_datetime(calendar_prices.date)), calendar_prices.holiday)).astype(float).interpolate(method ='pad', limit_direction ='forward', limit = 60))

holiday = calendar_prices[~calendar_prices.holiday.fillna('none').str.contains(r'(Observed)|none')].holiday.unique().tolist()#.unique()

In [ ]:
date_cols = ["Field_{}".format(i) for i in [5, 6, 7, 8, 9, 11, 15, 25, 32, 33, 35, 40]]
datetime_cols = ["Field_{}".format(i) for i in [1, 2, 43, 44]]
correct_dt_cols = ['Field_34', 'ngaySinh']
cat_cols = date_cols + datetime_cols + correct_dt_cols

# Normalize Field_34, ngaySinh
def ngaysinh_34_normalize(s):
    if s != s: return np.nan
    try: s = int(s)
    except ValueError: s = s.split(" ")[0]
    return datetime.strptime(str(s)[:6], "%Y%m")

# Normalize datetime data
def datetime_normalize(s):
    if s != s: return np.nan
    s = s.split(".")[0]
    if s[-1] == "Z": s = s[:-1]
    return datetime.strptime(s, "%Y-%m-%dT%H:%M:%S")

# Normalize date data
def date_normalize(s):
    if s != s: return np.nan
    try: t = datetime.strptime(s, "%m/%d/%Y")
    except: t = datetime.strptime(s, "%Y-%m-%d")
    return t

def process_datetime_cols(df):
    df[datetime_cols] = df[datetime_cols].applymap(datetime_normalize)  
    df[date_cols] = df[date_cols].applymap(date_normalize)
    df[correct_dt_cols] = df[correct_dt_cols].applymap(ngaysinh_34_normalize)

    # Some delta columns
    for i, j in zip('43 1 2'.split(), '1 2 44'.split()): df[f'DT_{j}_{i}'] = (df[f'Field_{j}'] - df[f'Field_{i}']).dt.seconds
    for i, j in zip('5 6 7 33 8 11 9 15 25 6 7 8 9 15 25 2'.split(), '6 34 33 40 11 35 15 25 32 7 8 9 15 25 32 8'.split()): 
        df[f'DT_{j}_{i}'] = (df[f'Field_{j}'] - df[f'Field_{i}']).dt.days
    
    # Age, month
    df['age'] = 2020 - pd.DatetimeIndex(df['ngaySinh']).year
    df['birth_month'] = pd.DatetimeIndex(df['ngaySinh']).month
    
    # Days from current time & isWeekday
    for col in cat_cols:
        name = col.split('_')[-1]
        df[f'is_WD_{name}'] = df[col].dt.dayofweek.isin(range(5))
        df[f'days_from_now_{name}'] = (datetime.now() - pd.DatetimeIndex(df[col])).days
        df[col] = df[col].dt.strftime('%m-%Y')
    
    # Delta for x_startDate and x_endDate
    for cat in ['F', 'E', 'C', 'G', 'A']:
        df[f'{cat}_startDate'] = pd.to_datetime(df[f"{cat}_startDate"], infer_datetime_format=True)
        df[f'{cat}_endDate'] = pd.to_datetime(df[f"{cat}_endDate"], infer_datetime_format=True)
        
        df[f'{cat}_start_end'] = (df[f'{cat}_endDate'] - df[f'{cat}_startDate']).dt.days
        
    for i, j in zip('F E C G'.split(), 'E C G A'.split()):
        df[f'{j}_{i}_startDate'] = (df[f'{j}_startDate'] - df[f'{i}_startDate']).dt.days
        df[f'{j}_{i}_endDate'] = (df[f'{j}_endDate'] - df[f'{i}_endDate']).dt.days
    
    temp_date = [f'{i}_startDate' for i in 'ACEFG'] + [f'{i}_endDate' for i in 'ACEFG']
    
    for col in temp_date:
        df[col] = df[col].dt.strftime('%m-%Y')
        
    for col in cat_cols + temp_date:
        df[col] = df[col].astype("category")
        
    return df

# Feature Selection




Research:
- [sklearn.feature_selection](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection)
- [2](https://www.kaggle.com/sz8416/6-ways-for-feature-selection) example selectkbes-RFE in kaggle
- [3](https://www.kaggle.com/dkim1992/feature-selection-ranking) chua doc
- [4](https://machinelearningmastery.com/feature-selection-machine-learning-python/) tong quat
- [sklearn.feature_selection](https://scikit-learn.org/stable/modules/feature_selection.html)
- [Cách tìm feature bằng cách chạy các linear reg](https://blog.datadive.net/selecting-good-features-part-iv-stability-selection-rfe-and-everything-side-by-side/) chua doc
- [6. Blog tổng hợp các method](https://mlwhiz.com/blog/2019/08/07/feature_selection/)

- Feature Selection: ở bước này tập trung các kĩ thuật chọn feature để tốt ưu mô hình
  - Chọn được feature tốt và tối ưu được bộ nhớ
  - Chọn được model parameters tốt nhất phù hợp với bộ features tốt nhất
  - Kết hợp với các kĩ thuật chia dataset (CV, leave-one)

- Tiêu chí đánh giá **metric of feature**
  - AUC: thể hiện khả năng dự đoán
  - Correlation: kiểm tra độ tương quan giữa feature với target hoặc với các feature quan trọng khác
  - Converage: kiểm tra null, null nhiều thì ít thông tin
  - Weighted of evidence, and information value:
    - $\ln \left(\frac{P(\text {Good})}{P(\text {Bad})}\right)$
    - \begin{array}{l}{\text { Information value }} \\ {\qquad \sum(P(G o o d)-P(B a d)) * \ln \left(\frac{P(G o d)}{P(B a d)}\right)}\end{array}



[Brute Force Approach](https://www.kdnuggets.com/2017/11/rapidminer-basic-concepts-feature-selection.html)
- Cách tiếp cận bằng các chạy thử thất cả các feature combination và so sánh trên các metrics of feature

[Feature Selection]
- Forwarding: ta bắt đầu với tập feature rỗng, sau đó lần lượt add thêm feature vào tập này. Nếu thấy performance của model tăng ta sẽ tiếp tục quá trình này, ngược lại sẽ dừng lại.
- Backwarding: ta bắt đầu với toàn bộ tập feature, sau đó lần lượt remove từng feature khỏi tập này. Nếu thấy performance của model tăng hoặc giảm không quá nhiều, ta sẽ tiếp tục quá trình này, ngược lại nếu performance bị drop quá mạnh sẽ dừng lại.
- Hybridge: kết hợp cả 2 hướng trên




### FS with Null Importances

Feature selection process using target permutation tests actual importance significance against the distribution of feature importances when fitted to noise (shuffled target).

The notebook implements the following steps :
- Create the null importances distributions : these are created fitting the model over several runs on a shuffled version of the target. This shows how the model can make sense of a feature irrespective of the target.
- Fit the model on the original target and gather the feature importances. This gives us a benchmark whose significance can be tested against the Null Importances Distribution
- For each feature test the actual importance:
    - Compute the probabability of the actual importance wrt the null distribution. I will use a very simple estimation using occurences while the article proposes to fit known distribution to the gathered data. In fact here I'll compute 1 - the proba so that things are in the right order.
    - Simply compare the actual importance to the mean and max of the null importances. This will give sort of a feature importance that allows to see major features in the dataset. Indeed the previous method may give us lots of ones.


- https://www.kaggle.com/sonannguyenngoc/tech-feature-selection-with-null-importances

### High correlation 

**High correlation features**
Có thể xóa bỏ bớt các cột có high correlate với nhau vì: 
- các cột có ý nghĩa như nhau với model
- có cơ hội học được từ các cột khác
- **quan trọng trong việc train NN** vì giảm được khối lượng dữ liệu

In [ ]:
# drop columns with correlation greater than 0.9
corr_matrix = df.drop(columns="isMale").corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
print("high correlation features", to_drop)
df.drop(columns=to_drop, inplace=True)

In [ ]:
#TESTING FOR MULTICOLLINEARITY:
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
#RUNNING FOR MULTIPLE REGRESSION:
# %%capture
#gather features
vif_test = df_train
vif_test = vif_test.drop('new_Active_Net.1', axis = 1)
to_drop = ['new_Leader_ActNet_ratio_6m','new_Leader_RegionCode','new_Leader_APE_ratio_3m','new_NewRe_APE_ratio_3m','new_CaseNet_SumLast3m','new_Leader_APE_ratio_6m','new_NewRe_Manpower_SumLas3p','new_Leader_Case_ratio_3m','new_AG_Case_ratio_3m','new_NewRe_Case_Net_SumLas3p']
vif_test = vif_test.drop(to_drop,axis =1)
features = "+".join(vif_test.drop('label_1',axis=1).columns)
# get y and X dataframes based on this regression:
y, X = dmatrices('label_1 ~' + features, df_train, return_type='dataframe')
vif = pd.DataFrame()
range = np.arange(0,X.shape[1],1)
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range]
vif["features"] = X.columns

col_vif = vif.drop(0).features
df_train = df_train.drop(to_drop,axis =1)
df_test = df_test.drop(to_drop,axis =1)
vif.sort_values('VIF Factor',ascending = False)


### WOE & IV
- https://www.one-tab.com/page/0V6bYIX4ShSQmuW4bpD7Zg

```vi du```:
- [woe - kalapa creditscr2nd](http://localhost:8888/lab/tree/DS/Project/Kalapa-CreditScor1st/readme.md)

In [ ]:
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [ ]:
## Kiem tra IV cua du lieu
cols = [x for x in df_model.columns.tolist() if x not in id_col + ['label','ID']]
final_iv, IV = data_vars(df_model[cols],df_model.label)
IV.sort_values(by=['IV'])

# Loc du lieu bang IV
thress_hold = 0.01
cols = IV[IV.IV > thress_hold].VAR_NAME.tolist()
df_model = df_model[ id_col + ['label'] + [x for x in df_model.columns.tolist() if x in cols]]

In [ ]:
'''
Chọn giá trị one hot bằng WOE
'''
cat_cols = df_model.select_dtypes(include=['object']).columns.tolist()

def onehot_encode(df, onehot_col, prefix = True):
    dummies_col = list()
    for c in onehot_col:
        print('> Get dummies with prefix {}'.format(c+'_'))
        if prefix:
            df_dummies = pd.get_dummies(df[c], prefix = c ) 
        else: 
            df_dummies = pd.get_dummies(df[c]) 
        dummies_col = dummies_col +(df_dummies.columns.to_list())
        df = pd.concat( [df, df_dummies] , axis = 1)
    #print('dummies col: ',dummies_col)
    return df, dummies_col

# điều chính IV_thresshold để chọn những cột sau khi one-hot có IV > IV_thresshold
cols = cat_cols
IV_thresshold = 0.01

for col in cols:
    tmp = df_model[[col] + ['label']]
    tmp, dummies_col = onehot_encode(tmp, [col], prefix = False)

    final_iv, IV = data_vars(tmp[dummies_col],df_model.label)
    #IV.sort_values(by=['IV'])
    highwoe_list = IV[IV.IV > IV_thresshold].VAR_NAME.tolist()
    if len(highwoe_list) > 0 :
        tmp[col] = np.where(tmp[col].isin(highwoe_list),tmp[col],'others')
        print("     > Number of categorical in {}: {}" .format(col,len(tmp[col].unique())))
        df_model[col] = tmp[col]
    else:
        print("     > All sub-categories less than thresshold: ", col)
        next
    #df_model[col] = tmp[col]

In [ ]:
'''
Bien doi gia tri cua CAT feature thanh gia tri tu WOE
'''

WOE_cat_col = ['a', 'b', 'c']

def woe_catfeat(df, WOE_cat_col, label):
    final_iv, IV = data_vars(df[WOE_cat_col], df[label])
    woe = pd.DataFrame()
    new_woe_col = list()
    
    for col in WOE_cat_col:
        l = []
        t_final_iv = final_iv[final_iv.VAR_NAME == col]
        print('> ',col)
        print(' no of bins: ',t_final_iv.shape[0])
        if t_final_iv.shape[0] <= 10:
            print(t_final_iv[['VAR_NAME','MIN_VALUE']])
        else:
            print(t_final_iv.sample(5)[['VAR_NAME','MIN_VALUE']])
        print('\n')
        for x in df[col]:
            l.append(t_final_iv[t_final_iv.MIN_VALUE == x].WOE.values[0])
        woe[col+'_woe'] = l
        new_woe_col.append(col+'_woe')
        
    return woe, new_woe_col

df, new_woe_cat = woe_catfeat(df, WOE_cat_col, 'label')

In [ ]:
'''
Bien doi gia tri cua NUMERIC feature thanh gia tri tu WOE
'''

WOE_numeric_col = ['a', 'b', 'c']

def woe_numericfeat(df, WOE_numeric_col, label):
    final_iv, IV = data_vars(df[WOE_numeric_col], df[label])
    woe = pd.DataFrame()
    new_woe_col = list()
    
    for col in WOE_numeric_col:
        l = []
        t_final_iv = final_iv[final_iv.VAR_NAME == col]
        print('> ',col)
        print(' no of bins: ',t_final_iv.shape[0])
        if t_final_iv.shape[0] <= 10:
            print(t_final_iv[['VAR_NAME','MIN_VALUE','MAX_VALUE']])
        else:
            print(t_final_iv.sample(5)[['VAR_NAME','MIN_VALUE','MAX_VALUE']])
        print('\n')
        for x in df[col]:
            l.append(t_final_iv[(t_final_iv.MIN_VALUE <= x)& (t_final_iv.MAX_VALUE >= x)].WOE.values[0])
        woe[col+'_woe'] = l
        new_woe_col.append(col+'_woe')
        
    return woe, new_woe_col

df, new_woe_cat = woe_catfeat(df, WOE_cat_col, 'label')

### **RFE**



[theory](https://www.kaggle.com/nroman/recursive-feature-elimination)

Trong mini course này, tôi sẽ áp dụng hướng “backwarding”. Các bước thực hiện như sau:

- Đặt: n là số lần lặp feature selection, k là số feature sẽ drop ở mỗi lần lặp, p là AUC sau mỗi lần train
Train model với XGboost
- Lấy kết quả feature important sắp xếp giảm dần và loại ra k feature có giá trị thấp nhất
- Lưu lại performance hiện tại để so sánh với performance tiếp theo.
- Nếu thấp hơn ngưỡng p sẽ dừng
- Tiếp tục quá trình selection

Tuỳ theo số lượng feature và cài đặt hyper-parameter của model thì thời gian sẽ nhanh chậm khác nhau.

Additional:
- Genetic algorithm for feature selection

Source:
- [1](https://ongxuanhong.wordpress.com/2019/04/17/data-science-mini-course/#more-15645)
- [2](https://towardsdatascience.com/feature-selection-in-python-recursive-feature-elimination-19f1c39b8d15) doc them
- https://www.kaggle.com/arthurtok/feature-ranking-rfe-random-forest-linear-models
- https://www.kaggle.com/sz8416/6-ways-for-feature-selection


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

#lr_best = LogisticRegressionCV
#lr_best = LogisticRegression(C=7.7, penalty='l2' ,class_weight='balanced')

In [ ]:
from sklearn.feature_selection import RFECV
# Create the RFE object and compute a cross-validated score.
# The "accuracy" scoring is proportional to the number of correct classifications
rfecv = RFECV(estimator=lr_best, step=1, cv=10, scoring='roc_auc')
rfecv.fit(X_train, y_train)

best_features = X_train.columns[rfecv.support_].tolist()

print("Optimal number of features: %d" % rfecv.n_features_)
print('Selected features: %s' % best_features)


# Plot number of features VS. cross-validation scores
plt.figure(figsize=(10,6))
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(np.arange(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

# Data Type Handling

### Under-sampling

In [ ]:
# under-sampling 7:3
sample = train[train.label_2==1].shape[0]/3*7
train_US73=pd.concat([train[train.label_2!=1].sample(int(sample)),train[train.label_2==1]], axis = 0, ignore_index=True)
print('sample shape',train_US73.shape)

# under-sampling 5:5
# sample = train[train.label_2==1].shape[0]/5*5
# train_US55=pd.concat([train[train.label_2!=1].sample(int(sample)),train[train.label_2==1]], axis = 0, ignore_index=True
# print('sample shape',train_US73.shape)

In [ ]:
#X = train[dummies_col+numeric_col]
X_train = train_US73[[x for x in train.columns if x not in ['label_2','First_month']]]
y_train = train_US73['label_2']
X_test = test[[x for x in train.columns if x not in ['label_2','First_month']]]
y_test = test['label_2']

### Over-sampling

#### RandomOverSampler

In [ ]:
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import make_classification
from sklearn.svm import LinearSVC

from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.base import BaseSampler

from sklearn.metrics import roc_auc_score, f1_score

In [ ]:
#X = train[dummies_col+numeric_col]
X_train = train[[x for x in train.columns if x not in ['label_2','First_month']]]
y_train = train['label_2']
X_test = test[[x for x in train.columns if x not in ['label_2','First_month']]]
y_test = test['label_2']

In [ ]:
model_smote = LogisticRegression()

pipe = make_pipeline(RandomOverSampler(sampling_strategy=1, random_state=0), model_smote)
pipe.fit(X_train, y_train)

In [ ]:
model_smote_predictions = pipe.predict_proba(X_test)
model_smote_pred_label = pipe.predict(X_test) 
model_smote_roc_score = roc_auc_score( y_test, model_smote_predictions[:,1])
model_smote_f1_score = f1_score(y_test, model_smote_pred_label)
print('random forest roc score on test: ', model_smote_roc_score)
print('random forest f1 score on test: ', model_smote_f1_score)

confu_matrix = confusion_matrix(y_test, model_smote_pred_label) 
sns.heatmap(confu_matrix , annot=True, fmt='d')
print(classification_report(y_test, model_smote_pred_label) )

#### SMOTE

In [ ]:
#X = train[dummies_col+numeric_col]
X_train = train[[x for x in train.columns if x not in ['label_2','First_month']]]
y_train = train['label_2']
X_test = test[[x for x in train.columns if x not in ['label_2','First_month']]]
y_test = test['label_2']

In [ ]:
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import (SMOTE, BorderlineSMOTE, SVMSMOTE, SMOTENC)
from imblearn.over_sampling import RandomOverSampler
from imblearn.base import BaseSampler

rf_clf = LogisticRegression()

smotes = {0 : 'SMOTE',
          1 : 'BorderlineSMOTE',
          2 : 'SVMSMOTE',
          3 : 'ADASYN'}


for i, sampler in enumerate((SMOTE(sampling_strategy = 1, random_state=0),
                BorderlineSMOTE(sampling_strategy = 1, random_state=0, kind='borderline-1'),
                SVMSMOTE(sampling_strategy = 1, random_state=0),
                ADASYN(sampling_strategy = 1, random_state=0))):
    pipe_line = make_pipeline(sampler, rf_clf)
    pipe_line.fit(X_train, y_train)
    rf_predictions = pipe_line.predict_proba(X_test)
    rf_pred_label = pipe_line.predict(X_test) 
    rf_roc_score = roc_auc_score(y_test, rf_predictions[:,1])
    rf_f1_score = f1_score(y_test, rf_pred_label)
    print('------------------------------------------------')
    print('SMOTE method: ', smotes[i])
    print('random forest roc score on test: ', rf_roc_score)
    print('random forest f1 score on test: ', rf_f1_score)

# Model Selection & Validation 

- https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

### Chiến thuật 1 - giảm variance trên k-fold
```Ví dụ```
- [kalapa scoring 1st - 2nd solution modelling](http://localhost:8888/lab/tree/DS/Project/Kalapa-CreditScor1st/2nd%20solution.ipynb)

```Tạo Baseline model```

Mình xin nói phần này trước phần Feature Engineering. Vì nó ảnh hưởng đến những việc mình làm ở các bước tiếp theo.
sau khi clean data, mình không làm feature gì thêm cả, đưa hết data vào model LGBM, chạy 5-folds sau đó submit. Sau đây là một số kết luận của mình từ baseline model.
- Không có correlation giữa validation và public LB.
- Validation rất quan trọng. Bởi vì nó ảnh hưởng đến việc bạn chọn hyper-parameters/feature engineering. Chỉ khi nào bạn setup đc một solid validation scheme, thì mới đến bước tiếp theo (Feature engineering, tunning, etc).
- Variance giữa các fold cực cao
- Mình thay đổi seed và kêt quả public LB thay đổi theo (nhưng k có variance). Thống kê trong 20 lần chạy với các seed khác nhau, variance giữa các seed cao.

**Tại sao lại variance cao?** - Theo quan điểm của mình, có thể:
- Do data. Nope, bạn k thể đổ lỗi cho data được. Đây là cái bạn phải đương đầu trong thực tiễn.
- Do model overfit.

**Variance cao thể hiện điều gì?**
Variance thể hiện tính ổn định của model. Variance cao tức là model không ổn định. Model không ổn định sẽ dẫn tới shakeup. Nó có thể tốt trên tập public test set, nhưng chưa chắc đã tốt trên tập test khác.
- Variance có quan trọng không?. Rất quan trọng.
- Tại sao?. Trong mô hình thực tế, data có thể thay đổi liên tục theo từng thời điểm. Tháng này Kalapa có 5triệu users, tháng sau có 15 triệu users. Distribution có thể bị skew liên tục. Công ty không cần bạn performance tốt trên 5tr user họ đang có (vì chính xác họ đã gán nhãn rồi), họ muốn kết quả tốt trên 10tr users sắp tới. Việc search seed minh nghĩ không phải một ý tưởng tốt trong bài này. Không thể để kết quả phụ thuộc quá nhiều vào con số hên xui. Vậy nên việc giữ cho variance càng thấp càng tốt sẽ giúp bạn tránh rủi ro hơn trọng kết quả cuối cùng.

```Feature engineering```

Không có gì đặc biệt trong phần này cả. Mình chỉ thêm một feature mới là mean target encoding.

```Feature selections```
Đây là phần quan trọng để giảm variance cũng như overfit.
Xuất phát ban đầu từ baseline, mình có 96 features. Sau mỗi lần chạy feature, mình tính feature importants và loại bỏ đi những feature yếu. Mỗi lần như vậy mình tỉa đi khoảng 5-10 features. Sau đó chạy lại lần nữa, và kết quả thấy variance giảm. Vòng loop được lặp lại cho đến khi variance không giảm được nữa thì thôi.
Cuối cùng mình có 31 features tất cả.

```Final Modeling```
light-gbm

Model của mình submit gồm 31 features trên. Mình chạy trong 20 seeds và lấy trung bình giữa các seed.
```
Gini for OOF: 0.2285142769658592
Average CV : 0.2343615754995229
STD CV : 0.01116168868867106
Gini for OOF: 0.23307624671948943
```

Cuối cùng mình giảm variance xuống được 0.01. Lúc này CV và LB có sự correlation và GAP không lớn. Có thể nói model cua mình ổn định và không overfit. public LB: 0.237, private LB: 0.257
Conclusion.

Mặc solution của đội mình không phải tốt nhất, nhưng mình tin nó sẽ giúp cho organizers build được model stable , ổn định nhất.

### Validation bằng cách search probabily ratio trên cut-off range

In [ ]:
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,roc_auc_score,roc_curve,auc,recall_score,f1_score
from sklearn.metrics import classification_report

def search_cutoff(model,x_test,y_test,search_range):
    #Logistics Regression:
    pred_prob= model.predict_proba(x_test)
#     search_range = np.arange(0,1,0.025)
    result = pd.DataFrame(columns = ['Thress_hold','accuracy','precision','recall','roc_auc','f1_score_poslabel','f1_score_neglabel'
                        ,'TP','TN','FN','FP'
                        ,'case','reduce','per_reduce','wrong','per_correct','send','per_send_correct'])

    for k in search_range:
        pred_var = np.where(pred_prob[:,1]>=k,1,0)#[1 if i > k else 0 for i in pred_prob[:,1]]
        confu_matrix = confusion_matrix(pred_var,y_test, labels=[1,0])
        result = pd.concat([result,pd.DataFrame([[
            k, accuracy_score(pred_var,y_test)
            ,precision_score(pred_var,y_test),recall_score(pred_var,y_test)
            ,accuracy_score(pred_var,y_test),f1_score(pred_var,y_test)
            ,f1_score(pred_var,y_test, pos_label=0)
            ,confu_matrix[0,0],confu_matrix[0,1],confu_matrix[1,0],confu_matrix[1,1]  
            ,len(pred_var),len(pred_var)-(confu_matrix[0,0]+confu_matrix[0,1]),1-round((confu_matrix[0,0]+confu_matrix[0,1])/len(pred_var),3)
            ,confu_matrix[1,0],round(confu_matrix[1,1]/(confu_matrix[1,0]+confu_matrix[1,1]),3)
            ,(confu_matrix[0,0]+confu_matrix[0,1]),round(confu_matrix[0,0]/(confu_matrix[0,0]+confu_matrix[0,1]),3)
        ]]
            ,columns = ['Thress_hold','accuracy','precision','recall','roc_auc','f1_score_poslabel','f1_score_neglabel'
                         ,'TP','TN','FN','FP'
                        ,'case','reduce','per_reduce','wrong','per_correct','send','per_send_correct'
                       ])])

    return result

'''
SHOW SOME IMPORTANT METRICS
'''
pred_prob = rf.predict_proba(x_test)
# pred_var = np.where(pred_prob[:,1]>=0.3,1,0)
pred_var = rf.predict(x_test)
print('Real label counts \n',y_test.value_counts())
print('  > label percent = ',1 - round(y_test.value_counts()[1]/y_test.size,3))
print('\n')

#precision_score(pred_var,y_test),recall_score(pred_var,y_test),
print('> Accuracy: ', accuracy_score(pred_var,y_test))
print('> Recall: ', recall_score(pred_var,y_test))
print('> Precision: ', precision_score(pred_var,y_test))
print('> F1 score: {}, F1 score pos: {} '.format(f1_score(pred_var,y_test),f1_score(pred_var,y_test, pos_label=0)))

print('> Consufion matric: \n',pd.DataFrame(confusion_matrix(pred_var,y_test, labels=[1,0]), index=['pred:1', 'pred:0'], columns=['true:1', 'true:0']))
print('> classification report: \n',classification_report(pred_var,y_test) )

search_range = np.arange(0,1,0.05)
search_cutoff(rf,x_test,y_test,search_range)

### Adversarial Validation
Adversarial Validation là một phương pháp giúp kiểm tra tính giống nhau (similarity) của distribution, tính chất (property) giữa tập train và tập test.

Việc thực hiện adversarial validation sẽ giúp ta biết nên trust Local cross-validation hay Leaderboard và có kế hoạch cụ thể cho việc validate model, tránh trường hợp bị overfit hay shakeup

Các bước thực hiện gồm:
- Drop các cột thông tin không cần thiết ở tập train và test (Ex: id). Cột target/label của tập train cũng được loại bỏ.
- Gán một label mới cho toàn bộ tập train là '1', label cho tập test là '0'.
- Dùng một model thông thường, không cần quá cồng kềnh (Randomforest, SVM, Xgboost, LightGBM,...) để phân loại đâu là sample của tập train (label=1) và đâu là data của tập test (label=0). Ở đây là bài toán binary classification nên hay dùng AUC để là metrics

Dựa vào giá trị AUC đo được ở trên, các trường hợp có thể xảy ra:
- AUC càng thấp (~0.5): Nghĩa là việc phân biệt sample của train và test là khó đối với model đó => Train và test giống nhau => Lúc này tỉ lệ shake-up sẽ thấp
- AUC càng cao (0.9): Nghĩa là việc phân biệt sample của train và test là dễ đối với model đó => Train và test khác nhau => Tỉ lệ shake-up cao. Bạn cần chọn cách chia validation sao cho nó giống với test nhất. Cách đơn giản nhất đó là chạy cross-validation với model thông thường trên. Với mỗi fold xác định xem sample nào bị miss-classified và đưa nó vào tập validation cho bài toán gốc

source:
- [code](https://www.kaggle.com/sonannguyenngoc/kalapa-adversarial-validation)


### Genetic algorithm
- [Explain 1](https://www.neuraldesigner.com/blog/genetic_algorithms_for_feature_selection)
- [Explain 2](https://towardsdatascience.com/feature-reduction-using-genetic-algorithm-with-python-403a5f4ef0c1)
- [Explain 3](http://dkopczyk.quantee.co.uk/genetic-algorithm/)

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import shutil

from sklearn.svm import SVR
from joblib import Parallel, delayed

# load train and valid
data_folder = 'competitionData'
train = pd.read_csv(os.path.join(data_folder, 'trainingDataDeflated.csv'))
valid = pd.read_csv(os.path.join(data_folder, 'validationDataDeflated.csv'))

# get training columns
training_columns = list(train.columns.values)
for column in ['ind', 'deck', 'nofGames', 'nOfPlayers', 'winRate']:
    training_columns.remove(column)
print("Total number of training columns: {}".format(len(training_columns)))

# set index for loc
train.set_index('ind', inplace=True)
valid.set_index('ind', inplace=True)

def load_dna_from_file(filename):
    dna = []
    f = open(filename)
    while True:
        line = f.readline().strip()
        if line == '':
            break
        # parse params
        params = line.split(";")
        indices = [int(item) for item in params[3].split(',') if item != '']
        # add information
        gene = dict()
        gene["epsilon"] = float(params[0])
        gene["c"] = float(params[1])
        gene["gamma"] = round(float(params[2]), 5)
        gene["indices"] = indices
        dna.append(gene)
    f.close()
    return dna

'''
save dna to file
'''
def save_dna_to_file(item):
    dna = item[0]
    fitness_score = item[1]
    filename = os.path.join("finalPopulation", "{}.txt".format(fitness_score))

    fout = open(filename, "w")
    for gene in dna:
        ind_text = ','.join(str(index) for index in gene["indices"])
        text = ';'.join([str(gene["epsilon"]), str(gene["c"]), str(gene["gamma"]), ind_text])
        fout.write(text + '\n')

    fout.close()

'''
initialize the first population, either randomly or from a file
'''
def init_population(population_size, from_files=False):
    pop = []

    # add good dna to the init population from previously tested good solutions
    if from_files == True:
        files = os.listdir("goodSolutions")
        for file in files:
            fitness_score = float(file[:-4])
            filename = os.path.join("goodSolutions", file)
            dna = load_dna_from_file(filename)
            pop.append([dna, fitness_score])

    num_of_random_dna = population_size - len(pop)
    for i in range(num_of_random_dna):
        dna = []
        for num_of_data in range(600, 1501, 100):
            gene = dict()
            gene["c"] = 5.0
            gene["gamma"] = 1.0/90
            gene["epsilon"] = 0.02
            gene["indices"] = sorted(random.sample(range(1, 100001), num_of_data))
            dna.append(gene)
        pop.append([dna, 0.0])

    min_score = min([item[1] for item in pop])
    return pop, min_score

'''
R2 predict evaluation
'''
def R2(x, y):
    return 1 - np.sum(np.square(x - y)) / np.sum(np.square(y - np.mean(y)))

'''
evaluate scores of a dna, sequential processing
'''
def fitness_sequential(dna, weight_train = 0.5):

    gene_valid_r2 = []
    gene_train_r2 = []

    for gene in dna:
        c = gene["c"]
        gamma = gene["gamma"]
        epsilon = gene["epsilon"]
        indices = gene["indices"]

        svr = SVR(kernel='rbf', gamma=gamma, C=c, epsilon=epsilon, shrinking=False)
        svr.fit(train.loc[indices][training_columns], train.loc[indices]["winRate"])
        valid_pred = svr.predict(valid[training_columns])
        valid_r2 = R2(valid_pred, valid["winRate"])
        gene_valid_r2.append(valid_r2)

        unselected_indices = list(set(range(1, 100001)) - set(indices))
        train_pred = svr.predict(train.loc[unselected_indices][training_columns])
        train_r2 = R2(train_pred, train.loc[unselected_indices]["winRate"])
        gene_train_r2.append(train_r2)

    fitness_valid = np.mean(gene_valid_r2)
    fitness_train = np.mean(gene_train_r2)
    fitness_score = (fitness_valid + weight_train * fitness_train) / (weight_train + 1.0)
    print("{},{},{}:{}".format(fitness_score, fitness_valid, fitness_train, gene_valid_r2 + gene_train_r2))
    return fitness_score

def train_and_valid(gene, using_train = False):
    c = gene["c"]
    gamma = gene["gamma"]
    epsilon = gene["epsilon"]
    indices = gene["indices"]

    svr = SVR(kernel='rbf', gamma=gamma, C=c, epsilon=epsilon, shrinking=False)
    svr.fit(train.loc[indices][training_columns], train.loc[indices]["winRate"])
    valid_pred = svr.predict(valid[training_columns])
    valid_r2 = R2(valid_pred, valid['winRate'])

    if using_train:
        unselected_indices = list(set(range(1, 100001)) - set(indices))
        train_pred = svr.predict(train.loc[unselected_indices][training_columns])
        train_r2 = R2(train_pred, train.loc[unselected_indices]["winRate"])
        return [valid_r2, train_r2]

    else:
        return valid_r2

'''
evaluate scores of a dna, parallel processing
'''
def fitness_parallel(dna, using_train = False, weight_train = 1.0 / 16, valid_threshold = 21.0):
    gene_r2 = Parallel(n_jobs=5, verbose=1)(delayed(train_and_valid)(gene, using_train) for gene in dna)
    if using_train:
        gene_valid_r2 = [item[0] for item in gene_r2]
        gene_train_r2 = [item[1] for item in gene_r2]
        fitness_valid = np.mean(gene_valid_r2)
        fitness_train = np.mean(gene_train_r2)
        fitness_score = (fitness_valid + fitness_train * weight_train) / (weight_train + 1.0)
        print("{},{},{}:{}".format(fitness_score, fitness_valid, fitness_train, gene_valid_r2 + gene_train_r2))
    else:
        fitness_score = np.mean(gene_r2)
        print("{}:{}".format(fitness_score, gene_r2))
    return fitness_score

'''
randomly replace a number of indices
no action is taken on epsilon, gamma, or c
'''
def mutate_gene(gene, percent_indices = 10):
    num_of_indices = len(gene["indices"])
    selected_mutation_part = random.randint(0, num_of_indices * 4)
    if selected_mutation_part < num_of_indices:
        # c varies from 0.00001 to 10.0
        gene["c"] = random.randint(1, 1000000) / 100000.0
    elif selected_mutation_part < num_of_indices * 2:
        # gamma varies from 0.01 to 10.0
        gene["gamma"] = round(random.randint(1, 9000000) / 900000.0, 5)
    elif selected_mutation_part <= num_of_indices * 3:
        # epsilon varies from 0.00001 to 1.0
        gene["epsilon"] = random.randint(1, 100000) / 100000.0
    else:
        indices = gene["indices"]
        num_of_removed_indices = int(percent_indices * num_of_indices / 100)
        removed_indices = random.sample(indices, num_of_removed_indices)

        remaining_indices = set(indices) - set(removed_indices)
        pool_indices = list(set(range(1, 100001)) - remaining_indices)
        new_indices = random.sample(pool_indices, num_of_removed_indices)

        gene["indices"] = list(remaining_indices) + new_indices
        assert len(gene["indices"]) == num_of_indices, "Missing indices in mutation"

'''
randomly select a gene for mutation
'''
def mutate_dna(dna, single_gene=True):
    if single_gene:
        selected_gene = random.randint(0, 9)
        mutate_gene(dna[selected_gene])
    else:
        for selected_gene in range(10):
            mutate_gene(dna[selected_gene])
    return dna

'''
perform crossover between 2 genes
'''
def crossover_gene(gene_x, gene_y):
    new_gene = dict()
    new_gene["c"] = random.sample([gene_x["c"], gene_y["c"]], 1)[0]
    new_gene["epsilon"] = random.sample([gene_x["epsilon"], gene_y["epsilon"]], 1)[0]
    new_gene["gamma"] = random.sample([gene_x["gamma"], gene_y["gamma"]], 1)[0]

    x_indices = set(gene_x["indices"])
    y_indices = set(gene_y["indices"])
    overlapping_indices = x_indices.intersection(y_indices)

    num_of_indices = len(x_indices)
    num_of_remaining_x_indices = int((num_of_indices - len(overlapping_indices)) / 2)
    num_of_remaining_y_indices = num_of_indices - len(overlapping_indices) - num_of_remaining_x_indices

    remaining_x_indices = list(x_indices - overlapping_indices)
    remaining_y_indices = list(y_indices - overlapping_indices)

    selected_x_indices = random.sample(remaining_x_indices, num_of_remaining_x_indices)
    selected_y_indices = random.sample(remaining_y_indices, num_of_remaining_y_indices)

    new_gene["indices"] = list(overlapping_indices) + selected_x_indices + selected_y_indices
    assert len(new_gene["indices"]) == num_of_indices, "Missing indices in crossover"

    return new_gene

'''
randomly select a gene and perform crossover on that gene
'''
def crossover_dna(dna_x, dna_y):

    # perform crossover on all genes
    new_dna = []
    for i in range(10):
        gene_x = dna_x[i]
        gene_y = dna_y[i]
        new_dna.append(crossover_gene(gene_x, gene_y))

    return new_dna

'''
select dna for new generation
'''
def select(population, population_size):
    if len(population) < population_size:
        return population

    prob_distribution = [item[1] for item in population]
    selected_indices = np.random.choice(range(len(population)), population_size, prob_distribution)
    removed_indices = [index for index in range(len(population)) if index not in selected_indices]

    # set new population based on selected indices
    new_population = []
    for index in selected_indices:
        new_population.append(population[index])

    # remove unselected_indices
    for index in sorted(removed_indices, reverse=True):
        del population[index]
    del population

    return new_population

'''
clone a gene
'''
def copy_gene(gene):
    new_gene = dict()
    new_gene["c"] = gene["c"]
    new_gene["gamma"] = gene["gamma"]
    new_gene["epsilon"] = gene["epsilon"]
    new_gene["indices"] = list(gene["indices"])
    return new_gene

'''
Clone a dna
'''
def copy_dna(dna):
    new_dna = []
    for gene in dna:
        new_gene = copy_gene(gene)
        new_dna.append(new_gene)
    return new_dna

'''
Select and update files for initial generation
'''
def update_files(population_size):
    new_solutions = os.listdir("finalPopulation")
    if len(new_solutions) == 0:
        return

    for file in new_solutions:
        if not os.path.exists(os.path.join("goodSolutions", file)):
            shutil.copy2(os.path.join("finalPopulation", file), "goodSolutions")
        os.remove(os.path.join("finalPopulation", file))

    all_solutions = os.listdir("goodSolutions")
    all_scores = [float(item[:-4]) for item in all_solutions]
    top_scores = sorted(all_scores, reverse=True)[:population_size]

    for item in all_solutions:
        if float(item[:-4]) not in top_scores:
            os.remove(os.path.join("goodSolutions", item))

if __name__ == "__main__":
    counter = 1
    while True:
        POPULATION_MAX = 21
        GENERATION_SIZE = 5

        # init population
        # tao ra population ban dau
        population, MIN_LOG_SCORE = init_population(POPULATION_MAX, from_files=True)
        print("Running times {}, min log score {}".format(counter, MIN_LOG_SCORE))

        # evolve
        for i in range(GENERATION_SIZE):
            new_solutions = []
            population_size = len(population)
            num_of_selected_parents = 15 # 3 * (population_size // 6)
            print("Generation {}, input size {}, num of selected parents {}".format(i + 1, population_size, num_of_selected_parents))

            # generate dna for mutation and crossover
            selected_indices = random.sample(range(population_size), num_of_selected_parents)

            for j in range(0, num_of_selected_parents, 3):
                # generate new child from mutation
                mutated_child = mutate_dna(copy_dna(population[selected_indices[j]][0]))
                new_solutions.append(mutated_child)

                mutated_child = mutate_dna(copy_dna(population[selected_indices[j + 1]][0]))
                new_solutions.append(mutated_child)

                mutated_child = mutate_dna(copy_dna(population[selected_indices[j + 2]][0]))
                new_solutions.append(mutated_child)

                # generate new children from crossover
                crossover_child = crossover_dna(population[selected_indices[j+1]][0], population[selected_indices[j+2]][0])
                new_solutions.append(crossover_child)

            # evaluate new children
            for dna in new_solutions:
                #fitness_score = fitness_sequential(dna)
                fitness_score = fitness_parallel(dna, using_train=True)
                if fitness_score > MIN_LOG_SCORE:
                    save_dna_to_file([dna, fitness_score])
                population.append([dna, fitness_score])

            # randomly select individuals from new population for the next generation
            population = select(population, POPULATION_MAX)

        # output final population
        '''
        for item in population:
            save_dna_to_file(item)
        '''

        # update good solutions for a new round
        update_files(POPULATION_MAX)
        counter += 1

